<a href="https://colab.research.google.com/github/SantiagoMengual/Politicas-Publicas/blob/main/TP_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabajo Final: Análisis geoespacial de la población que utilizó las bicicletas públicas para ir a su trabajo.

El siguiente trabajo busca conocer la ubicación y recorrido en la Ciudad de Buenos Aires de la población que utiliza la Ecobici para ir al trabajo.

Para dicho objetivo, trbajamos con la base de datos pública y abierta de los usuarios que utilizaron la Ecobici en el año 2023 y filtraremos para quedarnos con quienes la utilizaron entre als 6 AM y las 8 AM más de 8 minutos, cometiendo el salto intuitivo de inducir que dicha población la utiliza para trasladarse a su trabajo.

Mediante análisis de patrones en el espacio, clausterización y grafos se buscará llegar a nuevas conclusiones.

.Análisis de patrones en el espacio en base a la estación de origen de la población

.Clausterización del segmento de la población de la estación de origen con mayor peso en la muestra. Se buscará clausterizar en base al destino de dichos usuariois.

.Análisis de grafos: Trabajando con el mismo grupo poblacional, se buscará la ruta óptima para el viaje más repetido.





In [1]:

### Montamos drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# instalamos algunas librerias nuevas que vamos a usar
!pip install mapclassify folium pointpats contextily pygeos h3

# importamos librerias
import pandas as pd
import numpy as np

import geopandas as gpd

### Importar bibliotecas con alias es una práctica común en Python que ayuda a hacer el código más limpio y legible.

# PySAL va a ser una gran protagonista de acá en más
import libpysal
from pygeos import minimum_rotated_rectangle, from_shapely, to_shapely

### Importar las funciones directamente ahorra el llamar a las librerias.


import h3 #libreria para trabajar con hexagonos

from pointpats import centrography
from pointpats.centrography import hull, mbr, mean_center, weighted_mean_center, manhattan_median, std_distance,euclidean_median,ellipse

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as PolygonM #noten como usamos otro alias
from shapely.geometry import Polygon

from matplotlib.patches import Circle, Rectangle,Ellipse

import seaborn as sns
import contextily as ctx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 52.4 MB/s eta 0:00:00


In [ ]:
data_path = '/content/drive/MyDrive/Qgis 2024/Data/'


In [ ]:
viajes = pd.read_csv(data_path + 'trips_2023.csv')
viajes.sample(3)

In [ ]:
print(viajes.columns)

In [ ]:
# Eliminamos columas que no sirven
viajes = viajes.drop(['Unnamed: 0', 'modelo_bicicleta'], axis=1)


In [ ]:
# Nos quedamos con los viajes que se hayan realizado entre las 6 AM y las 8 AM

viajes_mañana = viajes[viajes['fecha_origen_recorrido'].str.contains('0[6-8]:')]


In [ ]:
## La población es todavía muy grande y poco definida

len(viajes_mañana)

In [ ]:
# Filtramos los viajes por duración: solo nos quedamos con los que hayan durado más de 480 segundos (8 minutos)

viajes_mañana['duracion_recorrido'] = viajes_mañana['duracion_recorrido'].str.replace(',', '').astype(int)

viajes_mañana = viajes_mañana[viajes_mañana['duracion_recorrido'] > 480]

In [ ]:
len(viajes_mañana)

In [ ]:
# Nos quedamos con los usuarios que hayan repetido origen y destino al menos una vez

origen_y_destino = viajes_mañana[viajes_mañana.duplicated(subset=['id_usuario', 'id_estacion_destino', 'id_estacion_origen'], keep=False)]



In [ ]:
# Creamos el factor de expansión

trip_counts = origen_y_destino.groupby(['id_usuario', 'id_estacion_origen', 'id_estacion_destino'])['id_usuario'].transform('count')

# Add the trip count as a new column called 'expansion_factor'
origen_y_destino['expansion_factor'] = trip_counts



In [ ]:
origen_y_destino.head(3)

In [ ]:
# Eliminamos los casos donde los nombres de las estaciones de origen y destino son iguales.
# Estos son casos donde el vijae enrealidad nunca salió de la estación.

origen_y_destino = origen_y_destino[origen_y_destino['nombre_estacion_origen'] != origen_y_destino['nombre_estacion_destino']]

In [ ]:

# Nos quedamos con las columnas que nos sirven y las reordenamos

origen = origen_y_destino[['id_usuario', 'nombre_estacion_origen', 'long_estacion_origen', 'lat_estacion_origen', 'expansion_factor']].copy()

origen.head(3)



In [ ]:
# Renombramos las columas

origen.rename(columns={
    'id_usuario': 'ID',
    'nombre_estacion_origen': 'NOMBRE',
    'long_estacion_origen': 'LONGITUD',
    'lat_estacion_origen': 'LATITUD',
    'expansion_factor': 'FACTOR_EXPANSION'
}, inplace=True)

origen.head(3)

In [ ]:
# Nos quedamos con los usuarios cuyo factor de expansión sea mayor a 10. Eliminamos usuarios cuyo peso no es significativo.

origen_filtrado = origen[origen['FACTOR_EXPANSION'] > 10]

origen_filtrado.head()

In [ ]:
len(origen_filtrado)

In [ ]:
# Eliminamos los duplicados, con el factor de expansión ya no nos sirven.

origen_filtrado = origen_filtrado.drop_duplicates(subset='ID', keep='first')

origen_filtrado.head()

In [ ]:
#Tenemos un grupo de estudio bien delimitado. Podemos comenzar a trabajar.

len(origen_filtrado)

In [ ]:
#Como primera aproximación generamos un mapa de calor

f, ax = plt.subplots(1, figsize=(9, 9))

sns.kdeplot(
    x="LONGITUD",
    y="LATITUD",
    data=origen_filtrado,
    n_levels= 5, #pueden ajustar este parametro
    fill=True,
    alpha=0.55,
    cmap="coolwarm",
)
ctx.add_basemap(
    ax, source=ctx.providers.CartoDB.Positron, crs="EPSG:4326"
)
ax.set_axis_on()

In [ ]:
orgien_filtrado = gpd.GeoDataFrame(origen_filtrado, geometry = gpd.GeoSeries.from_xy(x = origen.LONGITUD,y = origen.LATITUD, crs = 4326))


In [ ]:
## Para una mayor delimitación en el espacio, trabajamos con hexágonos

f, ax = plt.subplots(1, figsize=(12, 9))

hb = ax.hexbin(
    origen_filtrado["LONGITUD"],
    origen_filtrado["LATITUD"],
    gridsize=10,
    linewidths=0,
    alpha=0.5,
    cmap="coolwarm",
)

ctx.add_basemap(
    ax, source=ctx.providers.CartoDB.Positron,crs="EPSG:4326",
)
# Agregamos una barra de color a modo de leyenda
plt.colorbar(hb)
ax.set_axis_off()

In [ ]:
!pip install h3==3.7.6


import h3
h3.geo_to_h3?


In [ ]:
row = origen_filtrado.iloc[0]

In [ ]:
h3.geo_to_h3(lat = row.LATITUD, lng = row.LONGITUD, resolution = 8)

In [ ]:
origen_filtrado['h3'] = origen_filtrado.apply(lambda row: h3.geo_to_h3(lat = row.LATITUD, lng = row.LONGITUD, resolution = 8),axis=1)
origen_x_h3 = origen_filtrado.reindex(columns = ['h3','FACTOR_EXPANSION']).groupby('h3',as_index=False).sum()
geoms = origen_x_h3['h3'].map(lambda h: Polygon(h3.h3_to_geo_boundary(h,geo_json=True)[::-1]))
origen_x_h3 = gpd.GeoDataFrame(origen_x_h3, geometry = geoms, crs = 4326)
origen_x_h3.sample(3)

In [ ]:
origen_x_h3.explore(column = 'FACTOR_EXPANSION', scheme = 'fisherjenks', k = 5, tiles="CartoDB positron")

In [ ]:
# Averiguamos cual es la estación con h3 88c2e31101fffff

origen_x_h3_filtered = origen_filtrado[origen_filtrado['h3'] == '88c2e31101fffff']

origen_x_h3_filtered.head(180)

nombre_values = origen_x_h3_filtered['NOMBRE'].unique()

if len(nombre_values) == 1:
  print(f"All rows in 'origen_x_h3_filtered' share the same NOMBRE: {nombre_values[0]}")
else:
  print("Rows in 'origen_x_h3_filtered' do not share the same NOMBRE.")
  print(f"Unique NOMBRE values: {nombre_values}")

count_147_constitucion = origen_filtrado[origen_filtrado['NOMBRE'] == '147 - Constitución'].shape[0]

print(f"Number of rows with NOMBRE '147 - Constitución': {count_147_constitucion}")


Primera conclusión

La estación más utilizada como punto de partida con destino repetido es la estación de Constitución con 185 usuarios.

In [ ]:
#Graficamos las medidas de tendencia central. Primero las calculamos

mean_center = centrography.mean_center(origen_filtrado[["LONGITUD", "LATITUD"]])
med_center = centrography.euclidean_median(origen_filtrado[["LONGITUD", "LATITUD"]])


In [ ]:
joint_axes = sns.jointplot(x="LONGITUD", y="LATITUD", data=origen_filtrado, s=0.5);

# Agregamos esas medidas al plot
joint_axes.ax_joint.scatter(
    *mean_center, color="red", marker="x", s=50, label="Centro por la media"
)
joint_axes.ax_marg_x.axvline(mean_center[0], color="red")
joint_axes.ax_marg_y.axhline(mean_center[1], color="red")

joint_axes.ax_joint.scatter(
    *med_center,
    color="limegreen",
    marker="o",
    s=50,
    label="Centro por la mediana"
)
joint_axes.ax_marg_x.axvline(med_center[0], color="limegreen")
joint_axes.ax_marg_y.axhline(med_center[1], color="limegreen")

joint_axes.ax_joint.legend()
ctx.add_basemap(
    joint_axes.ax_joint, source=ctx.providers.CartoDB.Positron,
    crs="EPSG:4326",

)
joint_axes.ax_joint.set_axis_off()
plt.show()

In [ ]:
major, minor, rotation = centrography.ellipse(origen_filtrado[["LONGITUD", "LATITUD"]])

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
# Crear un simple scatter
ax.scatter(origen_filtrado["LONGITUD"], origen_filtrado["LATITUD"], s=0.75)
ax.scatter(*mean_center, color="red", marker="x", label="Centro por la media")
ax.scatter(
    *med_center, color="limegreen", marker="o", label="Centro por la mediana"
)

# Construir un objeto elipse de Matplotlib para representar esa dispersion
ellipse = Ellipse(
    xy=mean_center,  # el centro del elipse
    width=major * 2,  # centrography.ellipse da medio eje
    height=minor * 2,
    angle=np.rad2deg(
        rotation
    ),  # Los angulos estan en grados
    facecolor="none",
    edgecolor="red",
    linestyle="--",
    label="Std. Elipse",
)
ax.add_patch(ellipse)

ax.legend()
ctx.add_basemap(
    ax, source=ctx.providers.CartoDB.Positron,
    crs="EPSG:4326"
)
plt.show()

In [ ]:
origen_filtrado.head(3)

# **CLASE 3**

In [ ]:
# Creamos un nuevo objeto llamado constitucion_destino que solo se quede con los valores que cumplan en nombre_estacion_origen  : 147 - Constitución.
# Luego nos quedamos con las columas 'id_usuario', 'nombre_estacion_destino', 'long_estacion_destino', 'lat_estacion_destino', 'expansion_factor'.
# Luego renombramos estas columnas: id_usuario a ID, nombre_estacion_destino a NOMBRE, 'long_estacion_destino' a LONG, 'lat_estacion_destino'.

# Filtrar por 'nombre_estacion_origen' == '147 - Constitución'
constitucion_destino = origen_y_destino[origen_y_destino['nombre_estacion_origen'] == '147 - Constitución']

# Seleccionar columnas
constitucion_destino = constitucion_destino[['id_usuario', 'nombre_estacion_destino', 'long_estacion_destino', 'lat_estacion_destino', 'expansion_factor']]

# Renombrar columnas
constitucion_destino = constitucion_destino.rename(columns={
    'id_usuario': 'ID',
    'nombre_estacion_destino': 'NOMBRE',
    'long_estacion_destino': 'LONGITUD',
    'lat_estacion_destino': 'LATITUD',
    'expansion_factor': 'FACTOR_EXPANSION'
})

# Mostrar el nuevo DataFrame
constitucion_destino


In [ ]:
# prompt: Dame un caso de constitucion_destino que cumpla la condicion NOMBRE = 255 - BARRANCAS DE BELGRANO

# Find rows where NOMBRE is '255 - CONSTITUCION'
barrancas_rows = origen_filtrado[origen_filtrado['NOMBRE'] == '147 - Constitución']

# Display the matching rows (if any)
barrancas_rows

# **K-MEDIAS**

In [ ]:
X = constitucion_destino.loc[:,['LONGITUD','LATITUD']].values
w = constitucion_destino.FACTOR_EXPANSION.values
X

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Se crea el objeto para clusterizar
kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto")

# Se entrena o fitea con los datos y los pesos
wt_kmeansclus = kmeans.fit(X,sample_weight = w)

# Se toman las etiquetas de esa clasificacion
constitucion_destino['k'] = wt_kmeansclus.labels_

In [ ]:
constitucion_destino.reindex(columns = ['k','FACTOR_EXPANSION']).groupby('k').sum()['FACTOR_EXPANSION']


In [ ]:
geom = gpd.GeoSeries.from_xy(x=constitucion_destino.LONGITUD, y=constitucion_destino.LATITUD, crs=4326)
constitucion_destino = gpd.GeoDataFrame(constitucion_destino, geometry = geom, crs = 4326)

colors = ['red', 'blue', 'green', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'brown', 'black']

constitucion_destino.explore (column = 'k',
       categorical = True,
       cmap = colors)

In [ ]:
colors = ['red', 'blue', 'green', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'brown', 'black']

# Plot using explore with custom colors
constitucion_destino.explore(
    column='k',
    categorical=True,
    cmap=colors
)

# **DBSCAN**

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.007, min_samples=3)
wt_dbscanclus = dbscan.fit(X,sample_weight = w)
constitucion_destino['dbscan'] = wt_dbscanclus.labels_


In [ ]:
constitucion_destino.dbscan.value_counts(normalize=True)


In [ ]:

constitucion_destino.explore(column = 'dbscan', categorical = True)

In [ ]:
constitucion_destino.query("dbscan >= 0").explore(column = 'dbscan', categorical = True, tiles="CartoDB positron")

# GM

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# Se crea el objeto
gm = GaussianMixture(n_components= 5, random_state=0)

# Se fitea
wt_gmclus = gm.fit(X)

# En este caso sepuede predecir con datos nuevos o con los mismos datos
constitucion_destino['gm'] = gm.predict(X)

In [ ]:
constitucion_destino.explore(column = 'gm', categorical = True)

# **GRAFOS**

En esta sección se buscará graficar la distancia más corta entre las estaciones de Constitución y Barrancas de Belgrano. También se calculará su distancia en bicicleta.

Elegimos la estación Barrancas de Belgrano ya que en los gráficos anteriores podemos observar que es la que queda a mayor distancia de Constitución.

In [ ]:
!pip install osmnx mapclassify

import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import pandas as pd

# 1. Definir el área de estudio - Buenos Aires
city = "Buenos Aires, Argentina"

# Descargar el grafo de la red de calles de la ciudad
#
G = ox.graph_from_place(city, network_type='bike')

# Guardar el grafo en formato GraphML
#
ox.save_graphml(G, filepath=data_path+'buenos_aires_grafo.graphml')

# Leer desde el archivo
G = ox.load_graphml(data_path+'buenos_aires_grafo.graphml')

In [ ]:
fig, ax = ox.plot_graph(G)

In [ ]:
barrancas = (-58.448314, -34.559801	)
constitucion = (-58.380707,-34.626851	)

In [ ]:
nodo_barrancas = ox.distance.nearest_nodes(G, X = barrancas[0], Y = barrancas [1])
nodo_constitucion = ox.distance.nearest_nodes(G, X = constitucion[0], Y = constitucion[1])

In [ ]:
route = ox.shortest_path(G, nodo_constitucion, nodo_barrancas, weight="length")
fig, ax = ox.plot_graph_route(G, route, node_size=0)


In [ ]:
distance_meters = nx.shortest_path_length(G, nodo_constitucion, nodo_barrancas, weight='length')

# Convertir la distancia a kilómetros
distance_km = distance_meters / 1000

print(f"La distancia entre la estacion Constitucion y Barrancas de Belgrano es de aproximadamente {distance_km:.2f} km")